Something is wrong with the allocation of time to days after hump day (not equal distribution after that).

In [22]:
import operator
from datetime import date
import copy

In [2]:
class Task:
    def __init__(self,description,hoursNeeded,maxHoursPerDay,dueDate,daysLeft): 
        self.description = description
        self.hoursNeeded = hoursNeeded
        self.maxHoursPerDay = maxHoursPerDay
        self.dueDate = dueDate
        self.daysLeft = daysLeft

In [3]:
def addTask(description,hoursNeeded,maxHoursPerDay,dueDate):
    task = Task(description,hoursNeeded,maxHoursPerDay,dueDate,getTimeLeft(dueDate))
    tasks.append(task)

In [4]:
def getTimeLeft(date2):
    timeLeft = getTime(date2) - getTime(str(date.today())) + 1 
#    Get rid of the plus one to give a day of leeway for everything
#     if timeLeft == 0:
#         return 1
#     else:
    return timeLeft

In [5]:
def getTime(date): # Example date: "2021-11-17"
    
    year = date[:4]
    yearContribution = (int(year)-2021)*365 # NEEDS EDITING WHEN LEAP YEARS ARE COMING AROUND
    
    month = date[5:7]
    if month == "01":
        monthContribution = 0
    if month == "02":
        monthContribution = 31
    if month == "03":
        monthContribution = 60
    if month == "04":
        monthContribution = 91
    if month == "05":
        monthContribution = 121
    if month == "06":
        monthContribution = 152
    if month == "07":
        monthContribution = 182
    if month == "08":
        monthContribution = 213
    if month == "09":
        monthContribution = 244
    if month == "10":
        monthContribution = 274
    if month == "11":
        monthContribution = 305
    if month == "12":
        monthContribution = 335
        
    day = date[8:]
    dayContribution = int(day)-1
    
    daysSince2021 = yearContribution + monthContribution + dayContribution
    
    return daysSince2021

In [6]:
class Commitment:
    def __init__(self,description,hours,date): 
        self.description = description
        self.hours = hours
        self.date = date

In [7]:
def addComm(description,hours,date):
    comm = Commitment(description,hours,date)
    comms.append(comm)

Ultimately want schedule = {day1:(comms and times, tasks and times), day2:...}

1. Add all comms.
2. Figure out maxAvgHoursPerDay.
3. For the first task due, figure out the total hours (including commitments) due by its due date.
4. Saturate every day until humpDay, unless already saturated by existing comms. Be prepared to deal with more hours in commitments than maxAvgHoursPerDay.


In [8]:
def getSchedule(tasks):
    tasks = sortTasks(tasks)
    tasks = updateTasks(tasks)
    for task in tasks:
        print(task.description,"Days left:", task.daysLeft, "hoursNeeded:",task.hoursNeeded)
    for comm in comms:
        print(comm.description,f"hours: {comm.hours}",comm.date)
    tasksDue = []
    hoursDue = 0
    schedule = {}
    maxAvgHoursPerDay = 0
    humpDay = 0
    for day in range(1,tasks[-1].daysLeft+1):
        schedule[day] = {}
    for day in range(1,tasks[-1].daysLeft+1):
        newTaskWeights = {}
        newTasks = []
        newHours = 0
        for task in tasks:
            if task.daysLeft == day:
                tasksDue.append(task)
                hoursDue += task.hoursNeeded
                newTasks.append(task)
                newHours += task.hoursNeeded
                newDays = day - getFirstZero(schedule) + 1
        for newTask in newTasks:
            newTaskWeights[newTask] = newTask.hoursNeeded/newHours
#   print("\nDay = ",day)
#   print([(task.description,task.hoursNeeded,"Weight =",newTaskWeights[task]) for task in newTasks],"newH=",newHours)
        avgHoursPerDay = hoursDue/day
#         print("\nhoursDue,day,avgHoursPerDay : ",hoursDue,day,avgHoursPerDay,"\n")
        if avgHoursPerDay > maxAvgHoursPerDay:
            maxAvgHoursPerDay = avgHoursPerDay
            humpDay = day
#             print("humpDay = ",humpDay)
        for day2 in range(1,day+1):
            if avgHoursPerDay == maxAvgHoursPerDay:
                toAdd = (maxAvgHoursPerDay - getCurrentHours(schedule,day2))
                if toAdd > 0:
                    for newTask in newTasks:
                        schedule[day2][newTask.description] = toAdd*newTaskWeights[newTask]
            elif getCurrentHours(schedule,day2) == 0:
                for newTask in newTasks:
                    schedule[day2][newTask.description] = newHours/newDays * newTaskWeights[newTask]
    print("\nhumpDay =", humpDay)
    print("maxAvgHoursPerDay = ", maxAvgHoursPerDay)
    for day in schedule:
        print("\nDay", day,":\n")
        for taskDescription in schedule[day]:
            print(taskDescription+":",schedule[day][taskDescription])
#     return schedule

In [9]:
def sortTasks(tasks):
    sortedTasks = sorted(tasks, key=operator.attrgetter('daysLeft'))
    return sortedTasks

In [10]:
def updateTasks(tasks):
    Is = [] 
    for i in range(len(tasks)):
        try:
            tasks[i].daysLeft = getTimeLeft(tasks[i].dueDate)
            if tasks[i].daysLeft <= 0:
                remove = input("Remove "+ tasks[i].description+ "? (Y = yes, N = no)\n")
                if remove in "Yes":
                    Is.append(i)
                else:
                    tasks[i].dueDate = input("Enter new due date, in format yyyy-mm-dd: ")
                    tasks[i].daysLeft = getTimeLeft(tasks[i].dueDate)
        except IndexError:
            pass

    tasks2 = []
    for i in range(len(tasks)):
        if i not in Is:
            tasks2.append(tasks[i])
    return tasks2

In [11]:
def getFirstZero(schedule):
    for i in range(1,len(schedule)+1):
        if schedule[i] == {}:
            return i

In [12]:
def getCurrentHours(schedule,day2):
    if schedule[day2] == {}:
        return 0
    else:
        totalHours = 0
        workHours = schedule[day2].values()
        for hours in workHours:
            totalHours+=hours
        return totalHours

In [13]:
tasks = []
comms = []

In [14]:
addTask("Algebra HW",8,8,"2021-12-08")
addTask("ML Lecture",1,2,"2021-12-04")
addTask("Hex & Co Planning",7,6,"2021-12-03")
addTask("Research",5,5,"2021-12-07")
addTask("AI Research Cold Emails",5,3,"2021-12-03")
addTask("Redwood Interest Form",2,3,"2021-12-03")
addTask("Astrofest Poster Reimbursement",2,3,"2021-12-03")
# addTask("CC Reading 1",3,3,"2021-11-29")
addTask("QM HW",10,3,"2021-12-03")
addTask("Hex & Co Day",9,9,"2021-12-04")
addTask("ML HW",25,3,"2021-12-12")
addTask("CC Essay",7,3,"2021-12-03")
addTask("ML Studying",20,3,"2021-12-16")
addTask("QM Studying",20,3,"2021-12-21")
addTask("Algebra Studying",10,3,"2021-12-22")
addTask("AI Safety Fundamentals Application",1,1,"2021-12-15")

# tasks = sortTasks(tasks)
# tasks = updateTasks(tasks)

# addComm("CC Class",2,"2021-12-01")
# addComm("Algebra Class",2,"2021-12-01")

for task in tasks:
    print(task.description,task.daysLeft)
for comm in comms:
    print(comm.description,comm.date)

Algebra HW 6
ML Lecture 2
Hex & Co Planning 1
Research 5
AI Research Cold Emails 1
Redwood Interest Form 1
Astrofest Poster Reimbursement 1
QM HW 1
Hex & Co Day 2
ML HW 10
CC Essay 1
ML Studying 14
QM Studying 19
Algebra Studying 20
AI Safety Fundamentals Application 13


In [ ]:
getSchedule(tasks)

In [36]:
tasks = sortTasks(upDateTasks(tasks))
nDays = getNDays(tasks,comms)
schedule = createSchedule(nDays)
hded = getHDED(tasks,comms,nDays) # Hours due each day
print(hded)
previousHumpDay=0
while previousHumpDay < nDays:
    avgHPD,humpDay = getHumpDay(hded,previousHumpDay) # Average hours per day
    schedule = saturateSchedule(schedule,avgHPD,previousHumpDay,humpDay,tasks,comms)
    tasks,comms = getRemainingTasksAndComms(humpDay,tasks,comms)
    previousHumpDay = humpDay
print("\nhumpDay =", humpDay)
print("maxAvgHoursPerDay = ", maxAvgHoursPerDay)
for day in schedule:
    print("\nDay", day,":\n")
    for taskDescription in schedule[day]:
        print(taskDescription+":",schedule[day][taskDescription]) # task: hours of work for this task on this day

TypeError: object of type 'int' has no len()

In [22]:
def getNDays(tasks,comms):
    furthestDueDate = 0
    for task in tasks:
        if task.daysLeft > furthestDueDate:
            furthestDueDate = task.daysLeft
    for comm in comms:
        commTimeLeft = getTimeLeft(comm.date)
        if commTimeLeft > furthestDueDate:
            furthestDueDate = commTimeLeft
    return furthestDueDate

In [37]:
def createSchedule(nDays):
    schedule = {}
    for n in range(1,nDays+1):
        schedule[n]={}
    return schedule

In [27]:
def getHDED(tasks,comms,nDays):
    hded = [0]*nDays
    for task in tasks:
        daysLeft = task.daysLeft
        hded[daysLeft-1] += task.hoursNeeded
        
    for comm in comms:
        daysLeft = getTimeLeft(comm.date)
        hded[daysLeft-1] += comm.hours
    
    return hded

In [34]:
def getHumpDay(hded,previousHumpDay):
    remainingHDED = hded[previousHumpDay:]
    maxHPD = 0
    for i in range(len(remainingHDED)):
        totHDue = sum(remainingHDED[:i+1])
        avgHPD = totHDue/(i+1)
        if avgHPD>maxHPD:
            maxHPD = avgHPD
            humpDay = i+1
    return maxHPD, humpDay

In [ ]:
def saturateSchedule(schedule,avgHPD,previousHumpDay,humpDay,tasks,comms):
    tasks2 = []
    comms2= []
    for task in tasks:
        if task.daysLeft <= humpDay:
            task2 = Task(description,hoursNeeded,maxHoursPerDay,dueDate,getTimeLeft(dueDate)-previousHumpDay)
            tasks2.append(task2)
    for comm in comms:
        if getTimeLeft(comm.date) <= humpDay:
            comm2 = Commitment(comm.description,comm.hours,date)
            comms2.append(comm2)
    
    for comm in comms2:
        day = getTimeLeft(comm.date)
        schedule[day][comm] = comm.hours
        

In [ ]:
def getSchedule2(tasks):
    tasks = sortTasks(tasks)
    tasks = updateTasks(tasks)
    for task in tasks:
        print(task.description,"Days left:", task.daysLeft, "hoursNeeded:",task.hoursNeeded)
    for comm in comms:
        print(comm.description,f"hours: {comm.hours}",comm.date)
    tasksDue = []
    hoursDue = 0
    schedule = {}
    maxAvgHoursPerDay = 0
    humpDay = 0
    for day in range(1,tasks[-1].daysLeft+1):
        schedule[day] = {}
    for day in range(1,tasks[-1].daysLeft+1):
        newTaskWeights = {}
        newTasks = []
        newHours = 0
        for task in tasks:
            if task.daysLeft == day:
                tasksDue.append(task)
                hoursDue += task.hoursNeeded
                newTasks.append(task)
                newHours += task.hoursNeeded
                newDays = day - getFirstZero(schedule) + 1
        for newTask in newTasks:
            newTaskWeights[newTask] = newTask.hoursNeeded/newHours
#   print("\nDay = ",day)
#   print([(task.description,task.hoursNeeded,"Weight =",newTaskWeights[task]) for task in newTasks],"newH=",newHours)
        avgHoursPerDay = hoursDue/day
#         print("\nhoursDue,day,avgHoursPerDay : ",hoursDue,day,avgHoursPerDay,"\n")
        if avgHoursPerDay > maxAvgHoursPerDay:
            maxAvgHoursPerDay = avgHoursPerDay
            humpDay = day
#             print("humpDay = ",humpDay)
        for day2 in range(1,day+1):
            if avgHoursPerDay == maxAvgHoursPerDay:
                toAdd = (maxAvgHoursPerDay - getCurrentHours(schedule,day2))
                if toAdd > 0:
                    for newTask in newTasks:
                        schedule[day2][newTask.description] = toAdd*newTaskWeights[newTask]
            elif getCurrentHours(schedule,day2) == 0:
                for newTask in newTasks:
                    schedule[day2][newTask.description] = newHours/newDays * newTaskWeights[newTask]